# Evaluate the performance of a trained model

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import torch
from torch.utils.data import DataLoader
from torch.nn import DataParallel

from vos.models.STM import STM
from vos.algo.stm_train import STMAlgo
from vos.models.EMN import EMN
from vos.algo.emn_train import EMNAlgo
from vos.datasets.DAVIS import DAVIS_2017_TrainVal
from vos.utils.helpers import stack_images, stack_masks

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

Space-time Memory Networks: initialized.


In [2]:
davisroot = "/p300/videoObjSeg_dataset/DAVIS-2017-trainval-480p"
videod = DAVIS_2017_TrainVal(davisroot, mode= "val")
dataloader = DataLoader(videod)
print(len(dataloader))

30


In [3]:
model = DataParallel(STM())
algo = STMAlgo(eval_step_kwargs= dict(Mem_every= 5),)

# model = DataParallel(EMN())
# algo = EMNAlgo(eval_step_kwargs= dict(Mem_every= 5),)

model.cuda()
algo.initialize(model)
model.eval()

DataParallel(
  (module): STM(
    (Encoder_M): Encoder_M(
      (conv1_m): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (conv1_o): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (res2): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d

## Start evaluating and calculating the mean loss and performance

In [4]:
def run_test():
    loss, IoU, contour_acc = 0.,0.,0.
    IoU_each_frame = []
    for i, data in enumerate(dataloader):
        eval_info, extra_info = algo.eval(i, data)
        loss += eval_info.loss
        IoU += eval_info.IoU
        contour_acc += eval_info.contour_acc
        IoU_each_frame.append(extra_info["IoU_each_frame"])
        print(f"Evaluating at itereation: {i+1},", end= " ")
        print("IoU_this: {:.4f},".format(eval_info.IoU), end= " ")
        print("IoU: {:.4f},".format(IoU/(i+1)), end= " ")
        print("IoU_frame_ave: {:.4f},".format(np.nanmean(np.hstack(IoU_each_frame))), end= " ")
        print("ContourAcc_this: {:.4f},".format(eval_info.contour_acc), end= " ")
        print("ContourAcc: {:.4f},".format(contour_acc / (i+1)), end= " ")
        print("n_frames: {},".format(extra_info["IoU_each_frame"].shape[0]), end= "\r")
    return IoU / (i+1), np.nanmean(np.hstack(IoU_each_frame)), contour_acc / (i+1)

In [ ]:
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200319/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-4/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200322/synth5-5-0.05-5-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-4/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200319/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-1/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200319/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNEMN/big_objects-True1/b_size-1/run_0"
statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200326/img_res-384,384/NNSTM/b_size-4/pretrainTrue/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200326/img_res-384,384/NNSTM/b_size-4/pretrainFalse/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200327/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-4/run_0"
# statedict_root = "/p300/VideoObjSeg_data/local/video_segmentation/20200327/synth0.0-10.0-0.05-0.0-0.1/img_res-384,384/NNSTM/big_objects-True1/b_size-4/w_decay-0.0005/run_0"

IoU = 0
frames_IoU = 0
contour_acc = 0
threshold = 0.69
contour_threshold = 0.74
while IoU < threshold and frames_IoU < threshold and contour_acc < contour_threshold:
    torch.cuda.empty_cache()
    state_dict = torch.load(os.path.join(statedict_root, "params.pkl"))
#     state_dict = torch.load("/root/VideoObjSeg/data/weightfiles/STM_params_56.38DAVIS2017val.pkl")
    print("Train iteration: ", state_dict['itr_i'])
    model_state_dict = state_dict['model_state_dict']
    model.load_state_dict(model_state_dict)
    model.eval()
    IoU, frames_IoU, contour_acc = run_test()
    print("Testing result IoU: {:.5f}".format(IoU), "frames_IoU: {:.5f}".format(frames_IoU), "ContourAcc: {:.5f}".format(contour_acc), "###")


Train iteration:  12015


In [ ]:
dst = os.path.join(statedict_root, "params-{:.2f}-{:.2f}.pkl".format(frames_IoU*100, contour_acc*100))
print("Save to: ", dst)
torch.save(state_dict, dst)